In [1]:
!wget -q --show-progress -O Drivers.zip "https://www.dropbox.com/s/0vyzjcqsdl6cqi2/state-farm-distracted-driver-detection.zip?dl=0"

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
!unzip -q Drivers.zip

'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
!rm Drivers.zip

'rm' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
!unzip -q imgs.zip

'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
!rm imgs.zip

'rm' is not recognized as an internal or external command,
operable program or batch file.


In [6]:
import pandas as pd
driver_list=pd.read_csv('/content/driver_imgs_list.csv')


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\Hp\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\Hp\anaconda3\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
    app.start()
  File "C:\Users\Hp\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 736, in start
    self.io_loop.start()
  File "C:\Users\Hp\anaconda3\Lib\site-packages\tornado\

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\Hp\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\Hp\anaconda3\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
    app.start()
  File "C:\Users\Hp\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 736, in start
    self.io_loop.start()
  File "C:\Users\Hp\anaconda3\Lib\site-packages\tornado\

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\Hp\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\Hp\anaconda3\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
    app.start()
  File "C:\Users\Hp\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 736, in start
    self.io_loop.start()
  File "C:\Users\Hp\anaconda3\Lib\site-packages\tornado\

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\Hp\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\Hp\anaconda3\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
    app.start()
  File "C:\Users\Hp\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 736, in start
    self.io_loop.start()
  File "C:\Users\Hp\anaconda3\Lib\site-packages\tornado\

AttributeError: _ARRAY_API not found

FileNotFoundError: [Errno 2] No such file or directory: '/content/driver_imgs_list.csv'

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

image_path = '/content/test/img_7669.jpg'
image = mpimg.imread(image_path)
plt.imshow(image)
plt.axis('off')
plt.show()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255., validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    '/content/train',
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    '/content/train',
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


In [ ]:
from tensorflow.keras.layers import Dense,Flatten,Conv2D,MaxPooling2D,Dropout
from tensorflow.keras.models import Sequential

In [ ]:
model = Sequential()

model.add(Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(64,64,3)))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))  # Dropout to reduce overfitting
model.add(Dense(10, activation='softmax'))  # 10 categories

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
batch_size=32
steps_per_epoch=train_generator.n//batch_size
val_steps=val_generator.n//batch_size


In [ ]:
steps_per_epoch,val_steps

In [ ]:
model.fit(
    train_generator,
    #steps_per_epoch=steps_per_epoch,
    validation_data=val_generator,
    #validation_steps=val_steps,
    epochs=10
)

In [ ]:
import os

In [ ]:
test_dir = '/content/test'
test_images = [f for f in os.listdir(test_dir) if f.endswith('.jpg')]  # or .png, etc.
test_df = pd.DataFrame({'filename': test_images})

In [ ]:
# 1. Set up test generator (no data augmentation)
test_datagen = ImageDataGenerator(rescale=1./255.)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=test_dir,
    x_col='filename',
    y_col=None,
    target_size=(64, 64),
    batch_size=32,
    class_mode=None,
    shuffle=False
)


In [ ]:
label={'c0':'safe driving',
       'c1':'texting - right',
       'c2':'talking on the phone - right',
       'c3':'texting - left',
       'c4':'talking on the phone - left',
       'c5':'operating the radio',
       'c6':'drinking',
       'c7':'reaching behind',
       'c8':'hair and makeup',
       'c9':'talking to passenger'}

In [ ]:
import numpy as np

In [ ]:
# 2. Predict
predictions = model.predict(test_generator)
predicted_indices = np.argmax(predictions, axis=1)

# 3. Map class indices to original labels (e.g., 'c0', 'c1', ...)
class_indices = train_generator.class_indices
idx_to_class = {v: k for k, v in class_indices.items()}  # Reverse mapping
predicted_labels = [idx_to_class[i] for i in predicted_indices]
readable_labels = [label[class_code] for class_code in predicted_labels]


# 4. Get filenames (relative paths)
filenames = test_generator.filenames

# 5. Optional: Extract just the image names (without folders)
image_names = [os.path.basename(name) for name in filenames]

# 6. Save to CSV
results_df = pd.DataFrame({
    'filename': image_names,
    'predicted_label': predicted_labels,
    'description':readable_labels
    })

results_df.to_csv('test_predictions.csv', index=False)
print("Saved predictions to test_predictions.csv")

In [ ]:
test_pred=pd.read_csv('/content/test_predictions.csv')

In [ ]:
test_pred.head(5)